# Baseline with mse loss with wide cnn modified + adam
#### Convert histone mark signals and use deep CNN for training.

#### NOTE: Need to activate genomelake environment before this code. Simply type 'genomelake' in terminal.

In [10]:
%env CUDA_VISIBLE_DEVICES=1
import os, sys
sys.path.append("..")
import random
# custom file path package
from data import Data_Directories
# custom utility package
from utils.compute_util import *
# package for genomic data
from pybedtools import Interval, BedTool
from genomelake.extractors import ArrayExtractor, BigwigExtractor
# package for plotting
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats.stats import pearsonr,spearmanr
import tensorflow as tf

env: CUDA_VISIBLE_DEVICES=1


In [11]:
window_size = 2001
sample_num = 10000

In [12]:
# retrieve data
data = Data_Directories()
print data.intervals.keys()
print data.input_atac['day0'].keys()
print data.output_histone['day0'].keys()

['day6', 'day3', 'day0']
['100', '140']
['H3K27me3', 'H3K4me1', 'H3K27ac']


In [13]:
# get intervals for day0 data
day0_intervals = list(BedTool(data.intervals['day0']))
print '# of Intervals Extracted for day0: {}'.format(len(day0_intervals))

# of Intervals Extracted for day0: 267226


In [14]:
# create an ArrayExtractor for ATAC-seq for day0 with 140 base pairs
bw_140bp_day0 = ArrayExtractor(data.input_atac['day0']['140'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [15]:
# create a BigWigExtractor for histone makr 'H3K27ac' for day0
bw_histone_mark_day0 = BigwigExtractor(data.output_histone['day0']['H3K27ac'])
print 'Finished extracting bigwig for day0, 140bp'

Finished extracting bigwig for day0, 140bp


In [16]:
# normalize day0 intervals
normalized_day0_intervals = [normalize_interval(interval, window_size) for interval in day0_intervals if normalize_interval(interval, window_size)]
print 'Finished normalizing day0 intervals!'

Finished normalizing day0 intervals!


In [17]:
assert (len(day0_intervals)==len(normalized_day0_intervals))
print "Examples of original intervals"
print [(int(_interval.start)+int(_interval[-1]), [int(_interval.start), int(_interval.end)])
       for _interval in day0_intervals[:3]]
print "Examples of normalized intervals with window size of {}".format(window_size)
print [([int(_interval.start), int(_interval.end)])
       for _interval in  normalized_day0_intervals[:3]]

Examples of original intervals
[(123412027, [123411855, 123412989]), (123411941, [123411855, 123412989]), (131908564, [131908487, 131910071])]
Examples of normalized intervals with window size of 2001
[[123411027, 123413028], [123410941, 123412942], [131907564, 131909565]]


In [18]:
atac_seq_day0 = bw_140bp_day0(normalized_day0_intervals)
print atac_seq_day0.shape

(267226, 2001, 5)


In [19]:
#TODO: put this into utils if possible
def prune_invalid_intervals(intervals, bigwig_file):
    for _interval in intervals[:]:
        try:
            bigwig_file([_interval])
        except:
            intervals.remove(_interval)
            pass
        
print "Before pruning day0: {}".format(len(normalized_day0_intervals))
prune_invalid_intervals(normalized_day0_intervals, bw_140bp_day0)
print "After pruning day0: {}".format(len(normalized_day0_intervals))

Before pruning day0: 267226
After pruning day0: 267226


In [20]:
print "Dimension of ATAC-seq signal: {}".format(bw_140bp_day0(normalized_day0_intervals[:1]).shape)

Dimension of ATAC-seq signal: (1, 2001, 5)


In [21]:
print "Dimension of histone mark signal: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1]).shape)

Dimension of histone mark signal: (1, 2001)


In [22]:
# replace nan values with zeros and convert it to p-values
histone_mark_day0 = np.nan_to_num(bw_histone_mark_day0(normalized_day0_intervals))
print histone_mark_day0.shape

(267226, 2001)


In [23]:
histone_mark_day0 = np.expand_dims(histone_mark_day0, axis=2)
print histone_mark_day0.shape

(267226, 2001, 1)


In [24]:
print "Example histone mark signal"
print "\tRaw value: {}".format(bw_histone_mark_day0(normalized_day0_intervals[:1])[0][:5].reshape(-1))

Example histone mark signal
	Raw value: [ 0.01014  0.01014  0.01014  0.02435  0.02435]


In [25]:
from keras.layers import Input, Dense, Conv1D, Dropout, BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Model
from keras import optimizers
from keras import metrics
from keras import losses
from keras import backend as K
from keras.callbacks import Callback, TensorBoard, ReduceLROnPlateau, ModelCheckpoint

In [26]:
dropout_rate = 0.5
# parameters for first conv layer
hidden_filters_1 = 32
hidden_kernel_size_1 = window_size
# parameters for second conv layer
output_filters = 1
output_kernel_size = 32
# parameters for training
batch_size = 128
num_epochs = 150
evaluation_freq = 10

In [27]:
'''
Baseline CNN (based on KERAS functional API)
'''
inputs = Input(shape=(window_size, 5, ))
x = Conv1D(
    filters=hidden_filters_1,
    kernel_size=hidden_kernel_size_1,
    padding='same',
    activation='relu',
    strides=1,
    name='gen_conv1d_1')(inputs)
x = Dropout(dropout_rate,
            name='gen_dropout_1')(x)
# x = BatchNormalization(momentum=0.8)(x)

outputs = Conv1D(
    filters=output_filters,
    kernel_size=output_kernel_size,
    padding='same',
    activation='linear',
    strides=1,
    name='gen_conv1d_output'
    )(x)

model = Model(inputs=inputs, outputs=outputs)

In [28]:
# Print out model summary
print model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 2001, 5)           0         
_________________________________________________________________
gen_conv1d_1 (Conv1D)        (None, 2001, 32)          320192    
_________________________________________________________________
gen_dropout_1 (Dropout)      (None, 2001, 32)          0         
_________________________________________________________________
gen_conv1d_output (Conv1D)   (None, 2001, 1)           1025      
Total params: 321,217
Trainable params: 321,217
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
def pearson(y_true, y_pred):
    x = y_true
    y = y_pred
    mx = K.mean(x)
    my = K.mean(y)
    xm, ym = x-mx, y-my
    r_num = K.sum(np.multiply(xm,ym))
    r_den = K.sqrt(np.multiply(K.sum(K.square(xm)), K.sum(K.square(ym))))
    r = r_num / r_den
    r = K.maximum(K.minimum(r, 1.0), -1.0)
    return K.square(r)

In [30]:
# Helper functions for writing the scores into bigwig file
from itertools import izip
from itertools import groupby
import subprocess

def interval_key(interval):
    return (interval.chrom, interval.start, interval.stop)

def merged_scores(scores, intervals, merge_type):
    # A generator that returns merged intervals/scores
    # Scores should have shape: #examples x #categories x #interval_size
    # Second dimension can be omitted for a 1D signal
    signal_dims = scores.ndim - 1
    assert signal_dims in {1, 2}

    # Only support max for now
    assert merge_type == 'max'
    score_first_dim = 1 if signal_dims == 1 else scores.shape[1]

    dtype = scores.dtype

    sort_idx, sorted_intervals = \
        zip(*sorted(enumerate(intervals),
                    key=lambda item: interval_key(item[1])))
    sorted_intervals = BedTool(sorted_intervals)

    # Require at least 1bp overlap
    # Explicitly convert to list otherwise it will keep opening a file when
    # retrieving an index resulting in an error (too many open files)
    interval_clust = list(sorted_intervals.cluster(d=-1))
    for _, group in groupby(izip(sort_idx, interval_clust),
                            key=lambda item: item[1].fields[-1]):
        idx_interval_pairs = list(group)
        group_idx, group_intervals = zip(*idx_interval_pairs)

        if len(idx_interval_pairs) == 1:
            yield group_intervals[0], scores[group_idx[0], ...]
        else:
            group_chrom = group_intervals[0].chrom
            group_start = min(interval.start for interval in group_intervals)
            group_stop = max(interval.stop for interval in group_intervals)

            # This part needs to change to support more merge_types (e.g. mean)
            group_score = np.full((score_first_dim, group_stop - group_start),
                                  -np.inf, dtype)
            for idx, interval in idx_interval_pairs:
                slice_start = interval.start - group_start
                slice_stop = slice_start + (interval.stop - interval.start)
                group_score[..., slice_start:slice_stop] = \
                    np.maximum(group_score[..., slice_start:slice_stop],
                               scores[idx, ...])
            if signal_dims == 1:
                group_score = group_score.squeeze(axis=0)
            yield Interval(group_chrom, group_start, group_stop), group_score
            
def interval_score_pairs(intervals, scores, merge_type):
    return (izip(intervals, scores) if merge_type is None
            else merged_scores(scores, intervals, merge_type))

def _write_1D_deeplift_track(scores, intervals, file_prefix, merge_type='max',
                             CHROM_SIZES='/mnt/data/annotations/by_release/hg19.GRCh37/hg19.chrom.sizes'):
    assert scores.ndim == 2

    bedgraph = file_prefix + '.bedGraph'
    bigwig = file_prefix + '.bw'

    print 'Writing 1D track of shape: {}'.format(scores.shape)
    print 'Writing to file: {}'.format(bigwig)

    with open(bedgraph, 'w') as fp:
        for interval, score in interval_score_pairs(intervals, scores,
                                                    merge_type):
            chrom = interval.chrom
            start = interval.start
            for score_idx, val in enumerate(score):
                fp.write('%s\t%d\t%d\t%g\n' % (chrom,
                                               start + score_idx,
                                               start + score_idx + 1,
                                               val))
    print 'Wrote bedgraph.'

    try:
        output = subprocess.check_output(
            ['wigToBigWig', bedgraph, CHROM_SIZES, bigwig],
            stderr=subprocess.STDOUT)
        print 'wigToBigWig output: {}'.format(output)
    except subprocess.CalledProcessError as e:
        print 'wigToBigWig terminated with exit code {}'.format(
            e.returncode)
        print 'output was:\n' + e.output

    print 'Wrote bigwig.'

In [31]:
model_dir = os.path.join("models", "cnn_2000_new")
log_dir = os.path.join("logs", "cnn_2000_new")
srv_dir = os.path.join("/srv", "www", "kundaje", "jesikmin", "cnn_2000_new")
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
if not os.path.exists(srv_dir):
    os.makedirs(srv_dir)

In [32]:
# setting an adam optimizer with 1.0 clip norm
adam = optimizers.Adam(lr=1e-3, clipnorm=1.)

print "Compiling a model with adam optimizer"
model.compile(loss=losses.mean_squared_error,
              optimizer=adam,
              metrics=[pearson, metrics.mse, metrics.mae])

Compiling a model with adam optimizer


In [33]:
# CallBack: tensorboard with grad histogram
tensorboard = TensorBoard(log_dir=log_dir,
                          histogram_freq=1,
                          batch_size=batch_size,
                          write_grads=True)

In [34]:
# CallBack: reduce learning rate when validation loss meets plateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.96,
                              patience=5,
                              min_lr=1e-7)

In [35]:
# CallBack: evaluate model for every evaulation epoch
class EvaluateModel(Callback):
    def __init__(self, X_test, y_test):
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0

    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        # evaluate the model
        if self.epochs % evaluation_freq == 0:
            scores = model.evaluate(X_test, y_test)
            print zip(model.metrics_names, scores)
            test_prediction = model.predict(X_test)
            test_spearman_sum = 0.0
            for idx, _pred in enumerate(test_prediction):
                rho, _ = spearmanr(histone_mark_day0[10000+idx], _pred)
                test_spearman_sum += rho
            print "Test spearman: {}".format(test_spearman_sum * 1.0 / 1000)

In [36]:
# CallBack: store bigwig file for the best model
class SaveBigwig(Callback):
    def __init__(self, X_train, y_train, X_val, y_val, X_test, y_test):
        self.best_val_loss = float('Inf')
        self.best_epoch = -1
        self.X_train, self.y_train = X_train, y_train
        self.X_val, self.y_val = X_val, y_val
        self.X_test, self.y_test = X_test, y_test
        self.epochs = 0
    
    def on_epoch_end(self, batch, logs={}):
        self.epochs += 1
        cur_val_loss = logs['val_loss']
        if cur_val_loss < self.best_val_loss:
            self.best_val_loss = cur_val_loss
            self.best_epoch = self.epochs
            prediction = model.predict(X_train).reshape(int(sample_num*0.8), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[:int(sample_num*0.8)], os.path.join(srv_dir, 'train'))
            prediction = model.predict(X_val).reshape(int(sample_num*0.2), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[int(sample_num*0.8):sample_num], os.path.join(srv_dir, 'val'))
            prediction = model.predict(X_test).reshape(int(sample_num*0.1), window_size)
            _write_1D_deeplift_track(prediction,
                                     normalized_day0_intervals[sample_num:int(sample_num*1.1)], os.path.join(srv_dir, 'test'))
            f = open(os.path.join(srv_dir, 'meta.txt'), 'wb')
            f.write(str(self.epochs) + "  " + str(logs))
            f.close()

In [37]:
# CallBack: Save model checkpoint based on validation loss
checkpointer = ModelCheckpoint(os.path.join(model_dir, "best_model.h5"),
                               monitor='val_loss',
                               verbose=1,
                               save_best_only=True,
                               mode='min')

In [38]:
print "Fitting the model..."
train_index = int(sample_num*0.8)
val_index = sample_num
test_index = int(sample_num*1.1) 
X_train, y_train = atac_seq_day0[:train_index], histone_mark_day0[:train_index]
X_val, y_val = atac_seq_day0[train_index:val_index], histone_mark_day0[train_index:val_index]
X_test, y_test = atac_seq_day0[val_index:test_index], histone_mark_day0[val_index:test_index]

model.fit(atac_seq_day0[:sample_num],
          histone_mark_day0[:sample_num],
          batch_size=batch_size,
          epochs=num_epochs,
          validation_split=0.2,   
          #shuffle=True,
          callbacks=[tensorboard,
                     reduce_lr,
                     EvaluateModel(X_test, y_test),
                     checkpointer,
                     SaveBigwig(X_train, y_train, X_val, y_val, X_test, y_test)])

Fitting the model...
Train on 8000 samples, validate on 2000 samples
Epoch 1/150
7936/8000 [============================>.] - ETA: 0s - loss: 397.7688 - pearson: 0.1440 - mean_squared_error: 397.7688 - mean_absolute_error: 7.9988
Epoch 00001: val_loss improved from inf to 133.56077, saving model to models/cnn_2000_new/best_model.h5
Writing 1D track of shape: (8000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_2000_new/train.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (2000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_2000_new/val.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
Writing 1D track of shape: (1000, 2001)
Writing to file: /srv/www/kundaje/jesikmin/cnn_2000_new/test.bw
Wrote bedgraph.
wigToBigWig output: 
Wrote bigwig.
8000/8000 [==============================] - 168s 21ms/step - loss: 396.6375 - pearson: 0.1455 - mean_squared_error: 396.6375 - mean_absolute_error: 7.9981 - val_loss: 133.5608 - val_pearson: 0.1425 - 

7936/8000 [============================>.] - ETA: 0s - loss: 362.3485 - pearson: 0.2092 - mean_squared_error: 362.3485 - mean_absolute_error: 7.9769
Epoch 00012: val_loss did not improve
8000/8000 [==============================] - 17s 2ms/step - loss: 360.4872 - pearson: 0.2088 - mean_squared_error: 360.4872 - mean_absolute_error: 7.9589 - val_loss: 129.8220 - val_pearson: 0.1585 - val_mean_squared_error: 129.8220 - val_mean_absolute_error: 3.6965
Epoch 13/150
7936/8000 [============================>.] - ETA: 0s - loss: 360.1336 - pearson: 0.2120 - mean_squared_error: 360.1336 - mean_absolute_error: 7.8770
Epoch 00013: val_loss did not improve
8000/8000 [==============================] - 17s 2ms/step - loss: 360.4342 - pearson: 0.2122 - mean_squared_error: 360.4342 - mean_absolute_error: 7.8731 - val_loss: 130.5987 - val_pearson: 0.1553 - val_mean_squared_error: 130.5987 - val_mean_absolute_error: 3.7587
Epoch 14/150
7936/8000 [============================>.] - ETA: 0s - loss: 358.292

7936/8000 [============================>.] - ETA: 0s - loss: 354.5317 - pearson: 0.2271 - mean_squared_error: 354.5317 - mean_absolute_error: 7.9772
Epoch 00029: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 352.9278 - pearson: 0.2262 - mean_squared_error: 352.9278 - mean_absolute_error: 7.9606 - val_loss: 131.4655 - val_pearson: 0.1540 - val_mean_squared_error: 131.4655 - val_mean_absolute_error: 4.3563
Epoch 30/150
1000/1000 [==============================] - 1s 666us/steps: 354.0758 - pearson: 0.2259 - mean_squared_error: 354.0758 - mean_
[('loss', 199.20786364746093), ('pearson', 0.26734363746643064), ('mean_squared_error', 199.20786975097656), ('mean_absolute_error', 6.447439685821533)]
Test spearman: 0.315688502966

Epoch 00030: val_loss did not improve
8000/8000 [==============================] - 18s 2ms/step - loss: 353.9914 - pearson: 0.2255 - mean_squared_error: 353.9914 - mean_absolute_error: 8.0030 - val_loss: 135.1138 - val_pear

Epoch 46/150
7936/8000 [============================>.] - ETA: 0s - loss: 349.3282 - pearson: 0.2325 - mean_squared_error: 349.3282 - mean_absolute_error: 7.9355
Epoch 00046: val_loss did not improve
8000/8000 [==============================] - 17s 2ms/step - loss: 349.4650 - pearson: 0.2325 - mean_squared_error: 349.4650 - mean_absolute_error: 7.9328 - val_loss: 131.5234 - val_pearson: 0.1508 - val_mean_squared_error: 131.5234 - val_mean_absolute_error: 4.3367
Epoch 47/150
7936/8000 [============================>.] - ETA: 0s - loss: 351.2707 - pearson: 0.2305 - mean_squared_error: 351.2707 - mean_absolute_error: 7.9646
Epoch 00047: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 351.6580 - pearson: 0.2302 - mean_squared_error: 351.6580 - mean_absolute_error: 7.9693 - val_loss: 133.4320 - val_pearson: 0.1475 - val_mean_squared_error: 133.4320 - val_mean_absolute_error: 4.5482
Epoch 48/150
7936/8000 [============================>.] - ETA: 0s - 

Epoch 63/150
7936/8000 [============================>.] - ETA: 0s - loss: 345.3427 - pearson: 0.2350 - mean_squared_error: 345.3427 - mean_absolute_error: 7.9745
Epoch 00063: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 346.8021 - pearson: 0.2353 - mean_squared_error: 346.8021 - mean_absolute_error: 7.9837 - val_loss: 132.5795 - val_pearson: 0.1449 - val_mean_squared_error: 132.5795 - val_mean_absolute_error: 4.3252
Epoch 64/150
7936/8000 [============================>.] - ETA: 0s - loss: 347.0059 - pearson: 0.2362 - mean_squared_error: 347.0059 - mean_absolute_error: 7.9746
Epoch 00064: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 346.3576 - pearson: 0.2353 - mean_squared_error: 346.3576 - mean_absolute_error: 7.9754 - val_loss: 132.3859 - val_pearson: 0.1448 - val_mean_squared_error: 132.3859 - val_mean_absolute_error: 4.1553
Epoch 65/150
7936/8000 [============================>.] - ETA: 0s - 

1000/1000 [==============================] - 1s 654us/steps: 345.7548 - pearson: 0.2509 - mean_squared_error: 345.7548 - mean_
[('loss', 193.09801365661622), ('pearson', 0.27205539405345919), ('mean_squared_error', 193.09801461791992), ('mean_absolute_error', 6.1043274002075192)]
Test spearman: 0.314699651884

Epoch 00080: val_loss did not improve
8000/8000 [==============================] - 19s 2ms/step - loss: 344.9817 - pearson: 0.2516 - mean_squared_error: 344.9817 - mean_absolute_error: 7.9486 - val_loss: 133.2805 - val_pearson: 0.1469 - val_mean_squared_error: 133.2805 - val_mean_absolute_error: 4.6150
Epoch 81/150
7936/8000 [============================>.] - ETA: 0s - loss: 345.8297 - pearson: 0.2373 - mean_squared_error: 345.8297 - mean_absolute_error: 8.0190
Epoch 00081: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 344.9244 - pearson: 0.2366 - mean_squared_error: 344.9244 - mean_absolute_error: 8.0120 - val_loss: 132.6915 - val_pea

Epoch 97/150
7936/8000 [============================>.] - ETA: 0s - loss: 341.7919 - pearson: 0.2441 - mean_squared_error: 341.7919 - mean_absolute_error: 7.9980
Epoch 00097: val_loss did not improve
8000/8000 [==============================] - 17s 2ms/step - loss: 343.5314 - pearson: 0.2434 - mean_squared_error: 343.5314 - mean_absolute_error: 8.0135 - val_loss: 132.7107 - val_pearson: 0.1418 - val_mean_squared_error: 132.7107 - val_mean_absolute_error: 4.2692
Epoch 98/150
7936/8000 [============================>.] - ETA: 0s - loss: 342.4761 - pearson: 0.2421 - mean_squared_error: 342.4761 - mean_absolute_error: 7.8654
Epoch 00098: val_loss did not improve
8000/8000 [==============================] - 17s 2ms/step - loss: 342.4915 - pearson: 0.2422 - mean_squared_error: 342.4915 - mean_absolute_error: 7.8700 - val_loss: 133.6382 - val_pearson: 0.1414 - val_mean_squared_error: 133.6382 - val_mean_absolute_error: 4.4415
Epoch 99/150
7936/8000 [============================>.] - ETA: 0s - 

Epoch 114/150
7936/8000 [============================>.] - ETA: 0s - loss: 342.1828 - pearson: 0.2422 - mean_squared_error: 342.1828 - mean_absolute_error: 7.9276
Epoch 00114: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 341.5092 - pearson: 0.2416 - mean_squared_error: 341.5092 - mean_absolute_error: 7.9236 - val_loss: 133.4131 - val_pearson: 0.1377 - val_mean_squared_error: 133.4131 - val_mean_absolute_error: 4.2214
Epoch 115/150
7936/8000 [============================>.] - ETA: 0s - loss: 341.7750 - pearson: 0.2462 - mean_squared_error: 341.7750 - mean_absolute_error: 8.0153
Epoch 00115: val_loss did not improve
8000/8000 [==============================] - 17s 2ms/step - loss: 341.0608 - pearson: 0.2464 - mean_squared_error: 341.0608 - mean_absolute_error: 8.0096 - val_loss: 132.8884 - val_pearson: 0.1413 - val_mean_squared_error: 132.8884 - val_mean_absolute_error: 4.3745
Epoch 116/150
7936/8000 [============================>.] - ETA: 0s

Epoch 131/150
7936/8000 [============================>.] - ETA: 0s - loss: 341.0545 - pearson: 0.2480 - mean_squared_error: 341.0545 - mean_absolute_error: 7.8564
Epoch 00131: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 341.1305 - pearson: 0.2469 - mean_squared_error: 341.1305 - mean_absolute_error: 7.8605 - val_loss: 133.5933 - val_pearson: 0.1407 - val_mean_squared_error: 133.5933 - val_mean_absolute_error: 4.4088
Epoch 132/150
7936/8000 [============================>.] - ETA: 0s - loss: 341.5896 - pearson: 0.2567 - mean_squared_error: 341.5897 - mean_absolute_error: 7.8932
Epoch 00132: val_loss did not improve
8000/8000 [==============================] - 17s 2ms/step - loss: 341.3562 - pearson: 0.2561 - mean_squared_error: 341.3562 - mean_absolute_error: 7.8982 - val_loss: 133.3616 - val_pearson: 0.1385 - val_mean_squared_error: 133.3616 - val_mean_absolute_error: 4.2312
Epoch 133/150
7936/8000 [============================>.] - ETA: 0s

7936/8000 [============================>.] - ETA: 0s - loss: 340.8184 - pearson: 0.2476 - mean_squared_error: 340.8184 - mean_absolute_error: 7.8314
Epoch 00148: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 340.2210 - pearson: 0.2474 - mean_squared_error: 340.2210 - mean_absolute_error: 7.8317 - val_loss: 133.0116 - val_pearson: 0.1397 - val_mean_squared_error: 133.0116 - val_mean_absolute_error: 4.3517
Epoch 149/150
7936/8000 [============================>.] - ETA: 0s - loss: 340.0891 - pearson: 0.2507 - mean_squared_error: 340.0891 - mean_absolute_error: 7.8994
Epoch 00149: val_loss did not improve
8000/8000 [==============================] - 16s 2ms/step - loss: 340.0626 - pearson: 0.2498 - mean_squared_error: 340.0626 - mean_absolute_error: 7.9017 - val_loss: 133.9546 - val_pearson: 0.1341 - val_mean_squared_error: 133.9546 - val_mean_absolute_error: 4.0674
Epoch 150/150
1000/1000 [==============================] - 1s 670us/steps: 342.0